# Detailed execution time for cadCAD models

*Danilo Lessa Bernardineli*

---

This notebook shows how you can use metadata on PSUBs in order to do pre-processing on the simulations. We use two keys for flagging them: the `ignore` which indicates which PSUBs we want to skip, and the `debug`, which informs us what are the ones which we want to monitor the policies execution time.

In [9]:
from time import time
import logging
from functools import wraps
logging.basicConfig(level=logging.DEBUG)

def print_time(f):
  """

  """
  @wraps(f)
  def wrapper(*args, **kwargs):
      # Current timestep
      t = len(args[2])
      t1 = time()
      f_out = f(*args, **kwargs)
      t2 = time()
      text = f"{t}|{f.__name__} output (exec time: {t2 - t1:.2f}s): {f_out}"
      logging.debug(text)
      return f_out
  return wrapper

## Dependences

In [10]:
%%capture
!pip install cadcad 

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from cadCAD.configuration import Experiment
from cadCAD.configuration.utils import config_sim
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor

## Definitions

### Initial conditions and parameters

In [12]:
initial_conditions = {
    'prey_population': 100,
    'predator_population': 15
    }

params = {
    "prey_birth_rate": [1.0],
    "predator_birth_rate": [0.01],
    "predator_death_const": [1.0],
    "prey_death_const": [0.03],
    "dt": [0.1] # Precision of the simulation. Lower is more accurate / slower
}

simulation_parameters = {
    'N': 1,
    'T': range(30),
    'M': params
}

### Policies

In [13]:
def p_predator_births(params, step, sL, s):
  dt = params['dt']
  predator_population = s['predator_population']
  prey_population = s['prey_population']
  birth_fraction = params['predator_birth_rate'] + np.random.random() * 0.0002
  births =  birth_fraction * prey_population * predator_population * dt
  return {'add_to_predator_population': births}


def p_prey_births(params, step, sL, s):
  dt = params['dt']
  population = s['prey_population']
  birth_fraction = params['prey_birth_rate'] + np.random.random() * 0.1
  births =  birth_fraction * population * dt
  return {'add_to_prey_population': births}


def p_predator_deaths(params, step, sL, s):
  dt = params['dt']
  population = s['predator_population']
  death_rate = params['predator_death_const'] + np.random.random() * 0.005
  deaths = death_rate * population * dt
  return {'add_to_predator_population': -1.0 * deaths}


def p_prey_deaths(params, step, sL, s):
  dt = params['dt']
  death_rate = params['prey_death_const'] + np.random.random() * 0.1
  prey_population = s['prey_population']
  predator_population = s['predator_population']
  deaths = death_rate * prey_population * predator_population * dt
  return {'add_to_prey_population': -1.0 * deaths}

### State update functions

In [14]:
def s_prey_population(params, step, sL, s, _input):
    y = 'prey_population'
    x = s['prey_population'] + _input['add_to_prey_population']
    return (y, x)


def s_predator_population(params, step, sL, s, _input):
    y = 'predator_population'
    x = s['predator_population'] + _input['add_to_predator_population']
    return (y, x)

### State update blocks

In [15]:


partial_state_update_blocks = [
    { 
        'label': 'Predator dynamics',
        'ignore': False,
        'debug': True,
        'policies': {
            'predator_births': p_predator_births,
            'predator_deaths': p_predator_deaths
        },
        'variables': {
            'predator_population': s_predator_population            
        }
    },
    {
        'label': 'Prey dynamics',
        'ignore': True,
        'debug': True,
        'policies': {
            'prey_births': p_prey_births,
            'prey_deaths': p_prey_deaths
        },
        'variables': {
            'prey_population': s_prey_population
        }
    }
]

# Mantain only PSUBs which doesn't have the ignore flag
partial_state_update_blocks = [psub 
                               for psub in partial_state_update_blocks
                               if psub.get('ignore', False) == False]


# Only check the execution time for the PSUBs with the debug flag
for psub in partial_state_update_blocks:
    psub['policies'] = {label: print_time(f) for label, f in psub['policies'].items()}


### Configuration and Execution

In [16]:
sim_config = config_sim(simulation_parameters)

exp = Experiment()
exp.append_configs(sim_configs=sim_config, 
                   initial_state=initial_conditions,
                   partial_state_update_blocks=partial_state_update_blocks)


from cadCAD import configs
exec_mode = ExecutionMode()
exec_context = ExecutionContext(exec_mode.local_mode)
executor = Executor(exec_context=exec_context, configs=configs) 
(records, tensor_field, _) = executor.execute() 

DEBUG:root:1|p_predator_births output (exec time: 0.00s): {'add_to_predator_population': 1.5265279828492835}
DEBUG:root:1|p_predator_births output (exec time: 0.00s): {'add_to_predator_population': 1.5059518723732348}
DEBUG:root:1|p_predator_deaths output (exec time: 0.00s): {'add_to_predator_population': -1.5031816160768596}
DEBUG:root:1|p_predator_deaths output (exec time: 0.00s): {'add_to_predator_population': -1.5058029182164552}
DEBUG:root:2|p_predator_births output (exec time: 0.00s): {'add_to_predator_population': 1.5238729104761886}
DEBUG:root:2|p_predator_deaths output (exec time: 0.00s): {'add_to_predator_population': -1.5051680954024733}
DEBUG:root:3|p_predator_births output (exec time: 0.00s): {'add_to_predator_population': 1.5169348559862632}
DEBUG:root:3|p_predator_deaths output (exec time: 0.00s): {'add_to_predator_population': -1.5079580230502427}
DEBUG:root:4|p_predator_births output (exec time: 0.00s): {'add_to_predator_population': 1.505044567030721}
DEBUG:root:4|p_p

TypeError: cannot convert dictionary update sequence element #0 to a sequence

### Results

In [9]:
import plotly.express as px

In [10]:
df = pd.DataFrame(records)

fig = px.line(df,
              x=df.prey_population,
              y=df.predator_population,
              color=df.run.astype(str))

fig.show()